In [ ]:
from u_cmab import Fourier
from u_cmab import Static_UM
from u_cmab import plot_results
import simulation_code

import numpy as np
import math
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
window = 150

static_dataset_size = 2500
total_experiment_count = 30000
drift_moments = np.array([10000, 18000, 23000, 25000])

run_count = 10 #** 2

In [ ]:
line_args=[
    #{"color": "dodgerblue", "linewidth": 2, "zorder": 103, "label": "U-CMAB"}, 
    {"color": "firebrick", "linewidth": 2, "zorder": 0, "label": "Random Forest (ADWIN: 005; 2)"},
    {"color": "goldenrod", "linewidth": 2, "zorder": 0, "label": "Random Forest (ADWIN: 05; 2)"},
    {"color": "blueviolet", "linewidth": 2, "zorder": 0, "label": "Random Forest (ADWIN: 005; 02)"},
    {"color": "teal", "linewidth": 2, "zorder": 0, "label": "Random Forest (ADWIN: 05; 02)"},
    #{"color": "forestgreen", "linewidth": 2, "zorder": 103, "label": "CMAB"}
]
band_args=[
    #{"color": "turquoise", "alpha": .35, "linewidth":0, "zorder": 101}, 
    {"color": "deeppink", "alpha": .15, "linewidth":0, "zorder": 100},
    {"color": "lemonchiffon", "alpha": .15, "linewidth":0, "zorder": 100},
    {"color": "violet", "alpha": .15, "linewidth":0, "zorder": 100},
    {"color": "aqua", "alpha": .15, "linewidth":0, "zorder": 100},
    #{"color": "lime", "alpha": .25, "linewidth":0, "zorder": 100}
]
all_lines_args=[
    #{"color": "deepskyblue", "alpha": .1, "linewidth": .5}, 
    {"color": "crimson", "alpha": .1, "linewidth": .5},
    {"color": "crimson", "alpha": .1, "linewidth": .5},
    {"color": "crimson", "alpha": .1, "linewidth": .5},
    {"color": "crimson", "alpha": .1, "linewidth": .5},
    #{"color": "black", "alpha": .1, "linewidth": .5}
]
std_args=[
    #{"color": "deepskyblue", "linestyle": "--", "linewidth": 2, "dashes": [10, 10]}, 
    {"color": "crimson", "linestyle": "--", "linewidth": 2, "dashes": [10, 10]},
    {"color": "goldenrod", "linestyle": "--", "linewidth": 2, "dashes": [10, 10]},
    {"color": "goldenrod", "linestyle": "--", "linewidth": 2, "dashes": [10, 10]},
    {"color": "goldenrod", "linestyle": "--", "linewidth": 2, "dashes": [10, 10]},
    #{"color": "black", "linestyle": "--", "linewidth": 2, "dashes": [10, 10]}
]

stop_RP_args={"color": "tab:gray", "linewidth":2, "linestyle": "--", "dashes": [5,5], "alpha": .8}
stop_RP_text_args={"y": .95, "color": "tab:gray", "size": 19, "fontstyle": "italic"}
drift_line_args={"linestyle": ":", "color": "tab:green", "alpha": 1, "linewidth": 3}

In [ ]:
fourier_ids = np.empty([run_count, total_experiment_count])
fourier_reg_ids = np.empty([run_count, total_experiment_count])

static_ids_05_2 = np.empty([run_count, total_experiment_count])
static_params_05_2 = np.empty([run_count, 2])

static_ids_05_02 = np.empty([run_count, total_experiment_count])
static_params_05_02 = np.empty([run_count, 2])

static_ids_005_2 = np.empty([run_count, total_experiment_count])
static_params_005_2 = np.empty([run_count, 2])

static_ids_005_02 = np.empty([run_count, total_experiment_count])
static_params_005_02 = np.empty([run_count, 2])

for i in range(run_count):
    tau= (.45 - .25) * np.random.ranf() + .25
    ply_q = np.random.randint(6,10)
    K=np.random.normal(1,.5, ply_q+1)
    drift_rate=2.5# (3.0 - 2.0) * np.random.ranf() + 2.0
        #  the order of the Fourier approx. (O) is best chosen in 
        #  function of D as O and D affect the amount of frequency 
        #  vectors exponentially, causing a curse of dimensionality
        #  [Konindaris et al., 2011].
    D = np.random.randint(2,5)   
    O = 8-D 

    sim = simulation_code.Simulation(
        D=2, std=std, drift_moments=drift_moments, drift_rate=drift_rate, sudden_drift=True,
        base_functions=np.array([
            simulation_code.Sine_Base(f=4),
            simulation_code.Sine_Base(f=5)
        ]))
        
    fourier = Fourier(sim, order=5)
    fourier_id, fourier_dm, fourier_reg_id, tot_rew, exec_ac, tot_rew_reg, exec_ac_reg = fourier.run(
        epsilon=.1, 
        alpha=.005, 
        tau=tau, window=window, 
        lifetime=total_experiment_count)
    print("--Fourier {} Completed--".format(i+1))
    
    #static_PHalpha_PHlambda
    static_05_2 = Static_UM(RP_run_history=fourier.RP_run_history)
    up_05_2, static_id_05_2 = static_05_2.run( 
        static_dataset_size=static_dataset_size, 
        total_experiment_count=total_experiment_count, 
        tau=tau, window=window, PH_alpha=.05, PH_lambda=.2)
    
    static_05_02 = Static_UM(RP_run_history=fourier.RP_run_history)
    up_05_02, static_id_05_02 = static_05_02.run( 
        static_dataset_size=static_dataset_size, 
        total_experiment_count=total_experiment_count, 
        tau=tau, window=window, PH_alpha=.05, PH_lambda=.2)
    
    static_005_2 = Static_UM(RP_run_history=fourier.RP_run_history)
    up_005_2, static_id_005_2 = static_005_2.run( 
        static_dataset_size=static_dataset_size, 
        total_experiment_count=total_experiment_count, 
        tau=tau, window=window, PH_alpha=.005, PH_lambda=.2)
    
    static_005_02 = Static_UM(RP_run_history=fourier.RP_run_history)
    up_005_02, static_id_005_02 = static_005_02.run( 
        static_dataset_size=static_dataset_size, 
        total_experiment_count=total_experiment_count, 
        tau=tau, window=window, PH_alpha=.005, PH_lambda=.02)

    
    fourier_ids[i] = fourier_id
    fourier_reg_ids[i] = fourier_reg_id
    
    static_ids_05_2[i] = static_id_05_2
    static_params_05_2[i] = list(up_05_2.rand_search_.best_params_.values())
    
    static_ids_05_02[i] = static_id_05_02
    static_params_05_02[i] = list(up_05_02.rand_search_.best_params_.values())
    
    static_ids_005_2[i] = static_id_005_2
    static_params_005_2[i] = list(up_005_2.rand_search_.best_params_.values())
    
    static_ids_005_02[i] = static_id_005_02
    static_params_005_02[i] = list(up_005_02.rand_search_.best_params_.values())

In [ ]:
avg_ids = [
    np.mean(static_ids_005_2, axis=0),
    np.mean(static_ids_05_2, axis=0),
    np.mean(static_ids_005_02, axis=0),
    np.mean(static_ids_05_02, axis=0)
]
    
stds = [
    np.std(static_ids_005_2, axis=0),
    np.std(static_ids_05_2, axis=0),
    np.std(static_ids_005_02, axis=0),
    np.std(static_ids_05_02, axis=0)
]

In [ ]:
fig, ax = plot_results(avg_ids, stds, drift_moments, 
                 line_args=line_args, band_args=band_args, all_lines_args=all_lines_args, std_args=std_args,
                stop_RP=static_dataset_size, stop_RP_args=stop_RP_args, stop_RP_text_args=stop_RP_text_args, drift_line_args=drift_line_args)

In [ ]:
static_dataset_size_gd = 2500
total_experiment_count_gd = 20000
drift_over_time_gd = 30000
run_count_gd = 10 #** 2

In [ ]:
fourier_ids = np.empty([run_count, total_experiment_count])
fourier_reg_ids = np.empty([run_count, total_experiment_count])

static_ids_05_2 = np.empty([run_count, total_experiment_count])
static_params_05_2 = np.empty([run_count, 2])

static_ids_05_02 = np.empty([run_count, total_experiment_count])
static_params_05_02 = np.empty([run_count, 2])

static_ids_005_2 = np.empty([run_count, total_experiment_count])
static_params_005_2 = np.empty([run_count, 2])

static_ids_005_02 = np.empty([run_count, total_experiment_count])
static_params_005_02 = np.empty([run_count, 2])

for i in range(run_count):
    tau= (.45 - .25) * np.random.ranf() + .25
    ply_q = np.random.randint(6,10)
    K=np.random.normal(1,.5, ply_q+1)
    drift_rate=2.5
    
        #  the order of the Fourier approx. (O) is best chosen in 
        #  function of D as O and D affect the amount of frequency 
        #  vectors exponentially, causing a curse of dimensionality
        #  [Konindaris et al., 2011].
    D = np.random.randint(2,5)   
    O = 8-D 

    sim = simulation_code.Simulation(
        D=D, std=std, drift_over_time=drift_over_time_gd, drift_rate=drift_rate, sudden_drift=False,
        base_functions=np.array([
            simulation_code.Sine_Base(f=np.random.randint(3,6)), 
            simulation_code.Sine_Base(f=np.random.randint(4,7))
        ]))
        
    fourier = Fourier(sim, order=5)
    fourier_id, fourier_dm, fourier_reg_id, tot_rew, exec_ac, tot_rew_reg, exec_ac_reg = fourier.run(
        epsilon=.1, 
        alpha=.005, 
        tau=tau, window=window, 
        lifetime=total_experiment_count)
    print("--Fourier {} Completed--".format(i+1))
    
    #static_PHalpha_PHlambda
    static_05_2 = Static_UM(RP_run_history=fourier.RP_run_history)
    up_05_2, static_id_05_2 = static_05_2.run( 
        static_dataset_size=static_dataset_size, 
        total_experiment_count=total_experiment_count, 
        tau=tau, window=window, PH_alpha=.05, PH_lambda=.2)
    
    static_05_02 = Static_UM(RP_run_history=fourier.RP_run_history)
    up_05_02, static_id_05_02 = static_05_02.run( 
        static_dataset_size=static_dataset_size, 
        total_experiment_count=total_experiment_count, 
        tau=tau, window=window, PH_alpha=.05, PH_lambda=.2)
    
    static_005_2 = Static_UM(RP_run_history=fourier.RP_run_history)
    up_005_2, static_id_005_2 = static_005_2.run( 
        static_dataset_size=static_dataset_size, 
        total_experiment_count=total_experiment_count, 
        tau=tau, window=window, PH_alpha=.005, PH_lambda=.2)
    
    static_005_02 = Static_UM(RP_run_history=fourier.RP_run_history)
    up_005_02, static_id_005_02 = static_005_02.run( 
        static_dataset_size=static_dataset_size, 
        total_experiment_count=total_experiment_count, 
        tau=tau, window=window, PH_alpha=.005, PH_lambda=.02)

    
    fourier_ids[i] = fourier_id
    fourier_reg_ids[i] = fourier_reg_id
    
    static_ids_05_2[i] = static_id_05_2
    static_params_05_2[i] = list(up_05_2.rand_search_.best_params_.values())
    
    static_ids_05_02[i] = static_id_05_02
    static_params_05_02[i] = list(up_05_02.rand_search_.best_params_.values())
    
    static_ids_005_2[i] = static_id_005_2
    static_params_005_2[i] = list(up_005_2.rand_search_.best_params_.values())
    
    static_ids_005_02[i] = static_id_005_02
    static_params_005_02[i] = list(up_005_02.rand_search_.best_params_.values())

In [ ]:
avg_ids_G = [
    np.mean(static_ids_005_2, axis=0),
    np.mean(static_ids_05_2, axis=0),
    np.mean(static_ids_005_02, axis=0),
    np.mean(static_ids_05_02, axis=0),
]
    
stds_G = [
    np.std(static_ids_005_2, axis=0),
    np.std(static_ids_05_2, axis=0),
    np.std(static_ids_005_02, axis=0),
    np.std(static_ids_05_02, axis=0),
]

In [ ]:
fig_G, ax_G = plot_results(avg_ids_G, stds_G, np.array([]), 
                 line_args=line_args, band_args=band_args, all_lines_args=all_lines_args, std_args=std_args,
                stop_RP=static_dataset_size, stop_RP_args=stop_RP_args, stop_RP_text_args=stop_RP_text_args, drift_line_args=drift_line_args)